<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2024notebooks/2024notebooks/2024_0927pytorch_charRNN_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [青空文庫](https://www.aozora.gr.jp/) にあった雪女 (原作 小泉八雲) を用いた RNN のデモ

---
* date: 2024\_0927:
* author: 浅川伸一 educ0233@komazawa-u.ac.jp
---

In [ ]:
import torch
from torch import nn
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
device = 'cpu'

import sys
# 表示精度桁数の設定
import numpy as np
np.set_printoptions(suppress=False, formatter={'float': '{:6.3f}'.format})

# データの作成

In [ ]:
# https://www.aozora.gr.jp/cards/000258/files/50326_35772.html
txt = [ "武蔵の国のある村に茂作、巳之吉と云う二人の木こりがいた。 ",
       "この話のあった時分には、茂作は老人であった。",
        "そして、彼の年季奉公人であった巳之吉は、十八の少年であった。",
        "毎日、彼等は村から約二里離れた森へ一緒に出かけた。",
        "その森へ行く道に、越さねばならない大きな河がある。",
        "そして、渡し船がある。",
        "渡しのある処にたびたび、橋が架けられたが、その橋は洪水のあるたびごとに流された。",
        "河の溢れる時には、普通の橋では、その急流を防ぐ事はできない。",

        "　茂作と巳之吉はある大層寒い晩、帰り途で大吹雪に遇った。",
        "渡し場に着いた、渡し守は船を河の向う側に残したままで、帰った事が分った。",
        "泳がれるような日ではなかった。",
        "それで木こりは渡し守の小屋に避難した――避難処の見つかった事を僥倖に思いながら。",
        "小屋には火鉢はなかった。",
        "火をたくべき場処もなかった。",
        "窓のない一方口の、二畳敷の小屋であった。",
        "茂作と巳之吉は戸をしめて、蓑をきて、休息するために横になった。",
        "初めのうちはさほど寒いとも感じなかった。",
        "そして、嵐はじきに止むと思った。",

        "　老人はじきに眠りについた。",
        "しかし、少年巳之吉は長い間、目をさましていて、恐ろしい風や戸にあたる雪のたえない音を聴いていた。",
        "河はゴウゴウと鳴っていた。",
        "小屋は海上の和船のようにゆれて、ミシミシ音がした。",
        "恐ろしい大吹雪であった。",
        "空気は一刻一刻、寒くなって来た、そして、巳之吉は蓑の下でふるえていた。",
        "しかし、とうとう寒さにも拘らず、彼もまた寝込んだ。",

        "　彼は顔に夕立のように雪がかかるので眼がさめた。",
        "小屋の戸は無理押しに開かれていた。",
        "そして雪明かりで、部屋のうちに女、――全く白装束の女、――を見た。",
        "その女は茂作の上に屈んで、彼に彼女の息をふきかけていた、――そして彼女の息はあかるい白い煙のようであった。",
        "ほとんど同時に巳之吉の方へ振り向いて、彼の上に屈んだ。",
        "彼は叫ぼうとしたが何の音も発する事ができなかった。",
        "白衣の女は、彼の上に段々低く屈んで、しまいに彼女の顔はほとんど彼にふれるようになった、そして彼は――彼女の眼は恐ろしかったが――彼女が大層綺麗である事を見た。",
        "しばらく彼女は彼を見続けていた、――それから彼女は微笑した、そしてささやいた、――『私は今ひとりの人のように、あなたをしようかと思った。",
        "しかし、あなたを気の毒だと思わずにはいられない、――あなたは若いのだから。",
        "……あなたは美少年ね、巳之吉さん、もう私はあなたを害しはしません。",
        "しかし、もしあなたが今夜見た事を誰かに――あなたの母さんにでも――云ったら、私に分ります、そして私、あなたを殺します。",
        "……覚えていらっしゃい、私の云う事を』　そう云って、向き直って、彼女は戸口から出て行った。",
        "その時、彼は自分の動ける事を知って、飛び起きて、外を見た。しかし、女はどこにも見えなかった。",
        "そして、雪は小屋の中へ烈しく吹きつけていた。巳之吉は戸をしめて、それに木の棒をいくつか立てかけてそれを支えた。",
        "彼は風が戸を吹きとばしたのかと思ってみた、――彼はただ夢を見ていたかもしれないと思った。",
        "それで入口の雪あかりの閃きを、白い女の形と思い違いしたのかもしれないと思った。",
        "しかもそれもたしかではなかった。彼は茂作を呼んでみた。",
        "そして、老人が返事をしなかったので驚いた。",
        "彼は暗がりへ手をやって茂作の顔にさわってみた。",
        "そして、それが氷である事が分った。茂作は固くなって死んでいた。……"]

# データの加工

In [ ]:
chars = [' '] + list(sorted(set("".join(txt))))
n_vocab = len(chars)
chr2idx = { ch:i for i,ch in enumerate(chars) }
idx2chr = { i:ch for i,ch in enumerate(chars) }

maxlen = len(max(txt, key=len))
for i in range(len(txt)):
    while len(txt[i]) < maxlen:
        txt[i] += " "

# 入出力系列の定義
inp_seq, tgt_seq = [], []
for i in range(len(txt)):
    inp_seq.append(txt[i][:-1]) # 最後尾の要素を削除
    tgt_seq.append(txt[i][1:])  # 最先端の要素を削除
    print(f"{i:3d}:",
          f"入力系列: {inp_seq[i]}",
          f"出力系列: {tgt_seq[i]}")

inp_ids = [chr2idx[ch] for ch in inp_seq[i]]
tgt_ids = [chr2idx[ch] for ch in tgt_seq[i]]

dict_size = n_vocab
seq_len = maxlen - 1
batch_size = len(txt)

input_ids = torch.tensor(np.array(inp_ids), dtype=torch.int64)
target_ids = torch.Tensor(tgt_ids)

# ニューラルネットワークモデルの定義

In [ ]:
class RNN(nn.Module):
    def __init__(self,
                 n_vocab:int,
                 n_inp:int,
                 n_out:int,
                 n_hid:int,
                 n_layers:int,
                 device:str="cpu"):
        super().__init__()
        self.n_hid = n_hid
        self.n_layers = n_layers

        # ワンホットベクトルを埋め込み層へ変換
        self.embeddings = nn.Embedding(num_embeddings=n_vocab,
                                       embedding_dim=n_inp,
                                       device=device)

        # RNN 層本体
        self.rnn = nn.RNN(input_size=n_inp,
                          hidden_size=n_hid,
                          num_layers=n_layers,
                          batch_first=True,
                          nonlinearity='tanh',
                          bias=True,
                          bidirectional=False,
                          device=device)

        # 全結合層
        self.fc = nn.Linear(in_features=n_hid, out_features=n_out)

    def forward(self,
                inp:torch.Tensor,
                hidden:torch.Tensor=None,
                device=device):

        if hidden == None:  # 中間層の初期状態を設定
            hidden = self.init_hidden(batch_size=inp.size(0)).to(device)

        x = self.embeddings(inp).to(device)  # ワンホット表現を埋め込みベクトルへと変換
        out, hidden = self.rnn(x, hidden)    # リカレントニューラルネットワーク本体
        out = out.contiguous().view(-1, self.n_hid)  # 全結合層へ接続するために出力層のサイズに併せる
        out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size):
        # 中間層の初期状態を得る
        hidden = torch.zeros(batch_size, self.n_layers, self.n_hid)
        return hidden

# 訓練の実施

In [ ]:
# リカレントニューラルネットワークを引数を与えて実体化
model = RNN(n_vocab=dict_size, n_inp=128, n_hid=128, n_out=dict_size, n_layers=1, device="cpu") #device)
model.to(device)

# ハイパーパラメータの定義
n_epochs = 100
interval = 10
lr=0.01   # 学習率 learning ratio の省略形

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

model.train()
# Training Run
for epoch in range(n_epochs):
    optimizer.zero_grad() # 勾配の初期化
    input_ids.to(device)
    output,hidden = model(input_ids.unsqueeze(0))
    loss = criterion(output, target_ids.view(-1).long())
    loss.backward()  # 誤差逆伝播の実施
    optimizer.step() # 学習。すなわち結合係数の更新

    if epoch % interval == 0:
        print(f'エポック: {epoch:5d}/{n_epochs:5d}\t',
              f'Loss: {loss.item():.3f}')

# 結果の評価

In [ ]:
def predict(model, ch):
    # モデルと先頭の文字を受け取って，モデルの出力と最終時刻の中間層状態を返す
    chs = torch.from_numpy(np.array([[chr2idx[c] for c in ch]],dtype=int))
    chs.to(device)
    out, hid = model(chs)

    prob = nn.functional.softmax(out[-1], dim=0).data  # ソフトマックス関数により出力項目の確率を得る
    chr_idx = torch.max(prob, dim=0)[1].item()         # 最大確率を与える文字のインデックス番号を得る

    return idx2chr[chr_idx], hid

def sample(model, out_len, start='老'):
    # モデルと最大出力長と先頭の項目を与えて，生成された出力系列を返す
    model.eval() # eval mode
    chs = [ch for ch in start]
    size = out_len - len(chs)
    for ii in range(size):
        ch, h = predict(model, chs)
        chs.append(ch)

    return ''.join(chs)

In [ ]:
sample(model=model, out_len=30, start='茂')